<a href="https://colab.research.google.com/github/Jaideep0606/AI-Mafia---Machine-Learning/blob/master/ToDoManager_with_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import sqlite3
from getpass import getpass

class User:
    def __init__(self, username, password):
        self.username = username
        self.password = password

class Todo:
    def __init__(self, user_id, title, description, priority):
        self.user_id = user_id
        self.title = title
        self.description = description
        self.priority = priority
        self.complete = False

class TodoManager:
    def __init__(self):
        self.conn = sqlite3.connect('todo.db')
        self.create_tables()

    def create_tables(self):
        with self.conn:
            self.conn.execute('''
                CREATE TABLE IF NOT EXISTS users (
                    id INTEGER PRIMARY KEY,
                    username TEXT UNIQUE NOT NULL,
                    password TEXT NOT NULL
                )
            ''')
            self.conn.execute('''
                CREATE TABLE IF NOT EXISTS todos (
                    id INTEGER PRIMARY KEY,
                    user_id INTEGER NOT NULL,
                    title TEXT NOT NULL,
                    description TEXT,
                    priority INTEGER,
                    complete BOOLEAN NOT NULL CHECK (complete IN (0, 1)),
                    FOREIGN KEY (user_id) REFERENCES users (id)
                )
            ''')

    def register(self, username, password):
        with self.conn:
            try:
                self.conn.execute('INSERT INTO users (username, password) VALUES (?, ?)', (username, password))
                print("User registered successfully")
            except sqlite3.IntegrityError:
                print("Username already exists")

    def login(self, username, password):
        user = self.conn.execute('SELECT * FROM users WHERE username = ? AND password = ?', (username, password)).fetchone()
        if user:
            return user[0]
        else:
            print("Invalid credentials")
            return None

    def add_todo(self, user_id, title, description, priority):
        with self.conn:
            self.conn.execute('INSERT INTO todos (user_id, title, description, priority, complete) VALUES (?, ?, ?, ?, ?)',
                              (user_id, title, description, priority, False))
            print("Todo added successfully")

    def view_todos(self, user_id):
        todos = self.conn.execute('SELECT * FROM todos WHERE user_id = ?', (user_id,)).fetchall()
        for todo in todos:
            print(f"{todo[0]}: {todo[2]} - {todo[3]} (Priority: {todo[4]}, Complete: {todo[5]})")

    def update_todo(self, todo_id, title=None, description=None, priority=None, complete=None):
        with self.conn:
            todo = self.conn.execute('SELECT * FROM todos WHERE id = ?', (todo_id,)).fetchone()
            if not todo:
                print("Todo not found")
                return
            title = title or todo[2]
            description = description or todo[3]
            priority = priority or todo[4]
            complete = complete if complete is not None else todo[5]
            self.conn.execute('UPDATE todos SET title = ?, description = ?, priority = ?, complete = ? WHERE id = ?',
                              (title, description, priority, complete, todo_id))
            print("Todo updated successfully")

    def delete_todo(self, todo_id):
        with self.conn:
            self.conn.execute('DELETE FROM todos WHERE id = ?', (todo_id,))
            print("Todo deleted successfully")

def main():
    manager = TodoManager()
    print("Welcome to the Todo Manager CLI")
    while True:
        print("\n1. Register\n2. Login\n3. Exit")
        choice = input("Enter choice: ")
        if choice == '1':
            username = input("Enter username: ")
            password = getpass("Enter password: ")
            manager.register(username, password)
        elif choice == '2':
            username = input("Enter username: ")
            password = getpass("Enter password: ")
            user_id = manager.login(username, password)
            if user_id:
                while True:
                    print("\n1. Add Todo\n2. View Todos\n3. Update Todo\n4. Delete Todo\n5. Logout")
                    user_choice = input("Enter choice: ")
                    if user_choice == '1':
                        title = input("Enter title: ")
                        description = input("Enter description: ")
                        priority = int(input("Enter priority: "))
                        manager.add_todo(user_id, title, description, priority)
                    elif user_choice == '2':
                        manager.view_todos(user_id)
                    elif user_choice == '3':
                        todo_id = int(input("Enter todo ID: "))
                        title = input("Enter title (leave blank to keep current): ")
                        description = input("Enter description (leave blank to keep current): ")
                        priority = input("Enter priority (leave blank to keep current): ")
                        priority = int(priority) if priority else None
                        complete = input("Enter completion status (True/False, leave blank to keep current): ")
                        complete = True if complete.lower() == 'true' else False if complete.lower() == 'false' else None
                        manager.update_todo(todo_id, title, description, priority, complete)
                    elif user_choice == '4':
                        todo_id = int(input("Enter todo ID: "))
                        manager.delete_todo(todo_id)
                    elif user_choice == '5':
                        break
        elif choice == '3':
            break

if __name__ == "__main__":
    main()


Welcome to the Todo Manager CLI

1. Register
2. Login
3. Exit
Enter choice: 3
